In [1]:
# start by importing the modules we will need
import requests
import json  #we will use these later in retrieving geodata and plotting university locations using openstreetmaps
import folium
import numpy as np
import cufflinks as cf
import pandas as pd #These allow us to open our data in a pandas data frame
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
print("Ready to go to college?".title())

Ready To Go To College?


In [18]:
CENTER_US = (39.8333333,-98.585522)
#london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)

In [2]:
import sys
from sys import exit

help(exit)

Help on built-in function exit in module sys:

exit(...)
    exit([status])
    
    Exit the interpreter by raising SystemExit(status).
    If the status is omitted or None, it defaults to zero (i.e., success).
    If the status is an integer, it will be used as the system exit status.
    If it is another kind of object, it will be printed and the system
    exit status will be one (i.e., failure).



In [20]:
#accepted_states = ['Alabama','Alaska','Arizona','Arkansas' ,'California' ,'Colorado','Connecticut', 'Delaware','Florida','Georgia']
state = input('Enter State: ex. NewYork ')
try:
    state = pd.read_csv(state)
except FileNotFoundError:
    print('Please re-run and follow the correct format. Enter the state in title format. If the state has two words, ignore the space.')
    print("For example: Connecticut, NewYork, NewJersey, etc.")
    try:
        state = input('Enter State: ex. NewYork ')
        state = pd.read_csv(state)
    except FileNotFoundError:
        exit(0)
    

    #while True:
        #state = input('Enter State: ex. NewYork ')
        #if state is not in accepted_states:
            #break
        #else:
            #state = pd.read_csv(state)
#Add column based on size
state['size'] = np.nan
state['size'][state['Student population']>15000] = 'large'
state['size'][state['Student population']<=15000] = 'medium'
state['size'][state['Student population'] <=5000] = 'small'
state

display(state)
#filter by type of institution
college_types = list(state['Type'].value_counts().index)
i = 1
for types in college_types:
    print(i,types)
    i = i + 1
choice = input ('What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. ')
try:
    if choice == '':
        state2 = state
    else:
        choice = int(choice)#convert choice into an integer that corresponds to the menu options
        choice_value = college_types[choice-1]
        state2 = state[state['Type']==choice_value]
    display(state2)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state2 = state

#filter by surrounding area
surrounding_area = list(state2['Campus setting'].value_counts().index)
i = 1
for types in surrounding_area:
    print(i,types)
    i = i + 1
area = input ('What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.')
try:
    if area == '':
        state3 = state2
    else:
        area = int(area)
        setting_preference = surrounding_area[area-1]
        state3 = state2[state2['Campus setting']==setting_preference]
    display(state3)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state3=state2

#filter by size
college_sizes = list(state3['size'].value_counts().index)
i = 1
for types in college_sizes:
    print(i,types)
    i = i + 1
size_preference = input("How big do you want your university to be? Above are your options. Press enter if you have no preference.")
try:
    if size_preference == '':
        state4 = state3
    else:
        size_preference= int(size_preference) 
        size = college_sizes[size_preference -1]
        state4=state3[state3['size']==size]
    display(state4)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state4=state3

#filter by on campus housing
campus_housing = input('Do you want campus housing? Yes or no or type anything you do not have a preference. ')
campus_housing = campus_housing.capitalize()

if campus_housing == 'Yes':
    state5 = state4[state4['Campus housing'] == 'Yes']
    display(state5)
elif campus_housing == 'No':
    state5 = state4[state4['Campus housing'] == 'No']
    display(state5)
else: 
    state5 = state4
    display(state5)    


Enter State: ex. NewYork Iowa


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
0,Allen College,"1990 Heath St, Waterloo, Iowa 50703-1999",www.allencollege.edu,"4-year, Private not-for-profit",One but less than two years certificate|Associ...,City: Small,No,620.0,360.0,100%,NaN,Fall 2010,NaN,-,152798.0,3069100.0,small
1,Briar Cliff University,"3303 Rebecca Street, Sioux City, Iowa 51104",www.briarcliff.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,City: Small,Yes,1316.0,1117.0,44%,NaN,Fall 2010,"$18,818",-,152992.0,184600.0,small
2,Buena Vista University,"610 W 4th St, Storm Lake, Iowa 50588",www.bvu.edu,"4-year, Private not-for-profit",Bachelor's degree|Master's degree,Town: Remote,Yes,2114.0,1921.0,53%,NaN,Fall 2010,"$21,476",-,153001.0,184700.0,small
3,Central College,"812 University, Pella, Iowa 50219",www.central.edu,"4-year, Private not-for-profit",Bachelor's degree,Town: Distant,Yes,1248.0,1248.0,65%,NaN,Fall 2010,"$22,695",-,153108.0,185000.0,small
4,Clarke University,"1550 Clarke Dr, Dubuque, Iowa 52001-3198",www.clarke.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Postbacca...,City: Small,Yes,1043.0,801.0,61%,NaN,Fall 2010,"$23,787",-,153126.0,185200.0,small
5,Coe College,"1220 First Ave NE, Cedar Rapids, Iowa 52402-5092",www.coe.edu,"4-year, Private not-for-profit",Bachelor's degree,City: Midsize,Yes,1406.0,1406.0,67%,NaN,Fall 2010,"$21,870",-,153144.0,185400.0,small
6,Cornell College,"600 First St. SW, Mount Vernon, Iowa 52314-1098",www.cornellcollege.edu,"4-year, Private not-for-profit",Bachelor's degree,Town: Fringe,Yes,978.0,978.0,68%,26%,Fall 2010,"$27,915",-,153162.0,185600.0,small
7,Divine Word College,"102 Jacoby Dr SW, Epworth, Iowa 52045",www.dwci.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree,Rural: Distant,Yes,64.0,64.0,33%,44%,Fall 2010,"$6,917",-,153241.0,185800.0,small
8,Dordt College,"498 4th Ave NE, Sioux Center, Iowa 51250-1697",www.dordt.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,Town: Remote,Yes,1522.0,1454.0,69%,NaN,Fall 2010,"$24,109",-,153250.0,185900.0,small
9,Drake University,"2507 University Avenue, Des Moines, Iowa 50311...",www.drake.edu,"4-year, Private not-for-profit",Bachelor's degree|Master's degree|Post-master'...,City: Midsize,Yes,5001.0,3267.0,78%,NaN,Fall 2010,"$28,088",-,153269.0,186000.0,medium


1 4-year, Private not-for-profit
2 4-year, primarily associate's, Private for-profit
3 4-year, Private for-profit
4 4-year, Public
5 4-year, primarily associate's, Private not-for-profit
What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. 4


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
16,Iowa State University,"1750 Beardshear Hall, Ames, Iowa 50011-2103",www.iastate.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,36350.0,30671.0,74%,18%,Fall 2010,"$13,623",-,153603.0,186900.0,large
36,University of Iowa,"101 Jessup Hall, Iowa City, Iowa 52242-1316",www.uiowa.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,32011.0,24476.0,72%,21%,Fall 2010,"$14,161",-,153658.0,189200.0,large
37,University of Northern Iowa,"1227 W 27th St, Cedar Falls, Iowa 50614-0005",www.uni.edu,"4-year, Public",Bachelor's degree|Master's degree|Doctor's degree,City: Small,Yes,11905.0,10104.0,65%,14%,Fall 2010,"$14,831",-,154095.0,189000.0,medium


1 City: Small
What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
16,Iowa State University,"1750 Beardshear Hall, Ames, Iowa 50011-2103",www.iastate.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,36350.0,30671.0,74%,18%,Fall 2010,"$13,623",-,153603.0,186900.0,large
36,University of Iowa,"101 Jessup Hall, Iowa City, Iowa 52242-1316",www.uiowa.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,32011.0,24476.0,72%,21%,Fall 2010,"$14,161",-,153658.0,189200.0,large
37,University of Northern Iowa,"1227 W 27th St, Cedar Falls, Iowa 50614-0005",www.uni.edu,"4-year, Public",Bachelor's degree|Master's degree|Doctor's degree,City: Small,Yes,11905.0,10104.0,65%,14%,Fall 2010,"$14,831",-,154095.0,189000.0,medium


1 large
2 medium
How big do you want your university to be? Above are your options. Press enter if you have no preference.no
I am assuming you have no preference then. I guess we can move on.
Do you want campus housing? Yes or no or type anything you do not have a preference. sure


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
16,Iowa State University,"1750 Beardshear Hall, Ames, Iowa 50011-2103",www.iastate.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,36350.0,30671.0,74%,18%,Fall 2010,"$13,623",-,153603.0,186900.0,large
36,University of Iowa,"101 Jessup Hall, Iowa City, Iowa 52242-1316",www.uiowa.edu,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Small,Yes,32011.0,24476.0,72%,21%,Fall 2010,"$14,161",-,153658.0,189200.0,large
37,University of Northern Iowa,"1227 W 27th St, Cedar Falls, Iowa 50614-0005",www.uni.edu,"4-year, Public",Bachelor's degree|Master's degree|Doctor's degree,City: Small,Yes,11905.0,10104.0,65%,14%,Fall 2010,"$14,831",-,154095.0,189000.0,medium


In [23]:
school_data= pd.DataFrame({'Name':state4['Name'],
              'Location':state4['Address'],'Website': state4['Website']})
display(school_data)
#list(school_data)

#created a new data frame with the information we want. 



def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in school_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


,Location,Name,Website
16,"1750 Beardshear Hall, Ames, Iowa 50011-2103",Iowa State University,www.iastate.edu
36,"101 Jessup Hall, Iowa City, Iowa 52242-1316",University of Iowa,www.uiowa.edu
37,"1227 W 27th St, Cedar Falls, Iowa 50614-0005",University of Northern Iowa,www.uni.edu


In [15]:


def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords

for index, each_school in state.iterrows():
    location = each_school['Address']
    coords = get_coordinates_using_google(location)
    name = str(each_school['Name'])
    website = str(each_school['Website'])
    #print(name, website, coords)
    pos = (coords['lat'],coords['lng'])
    message = "%s: %s" % (name,website)
    i=i+1
    marker = folium.CircleMarker(location=pos,
                                popup=message,
                                color='#3333cc'
                                )
    map.add_children(marker)
map   


IndexError: list index out of range